<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/sketch/justin/COMPAREwith2019_Clean_CH_2020_Social_Distancing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clean Chicago 2020 Social Distancing Dataset

**Instructions:**

1. Mount the google drive to access the data by following **Steps to Mount the Drive**.
2. Provide the drive file path to the data, city name, and year in the fourth code cell. 

**Steps to Mount the Drive:**

1. Execute the second code cell.
2. There will be a link to follow in order to authorize the google account for drive. Go to that link.
3. A code to authorize the google account will be generated. Copy the code generated.
4. Go back to the cell where the process of mounting the drive is running. Paste the generated code from step 3 to the text box in the cell and press enter.

In [2]:
pip install geopandas

     |████████████████████████████████| 1.0MB 4.4MB/s 
     |████████████████████████████████| 15.3MB 383kB/s 
     |████████████████████████████████| 6.5MB 40.4MB/s 


In [3]:
import os
import geopandas
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

import altair as alt
from vega_datasets import data

# # Allow altair to visualize large datasets
# alt.data_transformers.disable_max_rows()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
city = 'Chicago'
city_directory = 'ch'
year = '2020'

# Provide a list of CSV file paths to read
path = Path('drive/MyDrive/big-data-project/data/clean-data/')
path_list = [path/city_directory/'social'/year]

In [6]:
# # Chicago Counties
# fips_ch = ['17031','17037','17043','17063','17091','17089','17093','17111','17197','17097','18073','18089','18111','18127','55059']

# # New York Counties
# fips_ny = ['36061','36047','36005','36085','36081']

# # LA Counties
# fips_la = ['06037','06059']

# path = Path('/Users/justinsnider/nyu-big-data/project/clean-data/')
# dir_list_la = [path/'la'/'social'/'2019',
#                path/'la'/'social'/'2020']

# dir_list_ny = [path/'ny'/'social'/'2019',
#                path/'ny'/'social'/'2020']

# dir_list_ch = [path/'ch'/'social'/'2019',
#                path/'ch'/'social'/'2020']

# dir_list = [path/'la'/'social'/'2019',
#             path/'la'/'social'/'2020',
#             path/'ny'/'social'/'2019',
#             path/'ny'/'social'/'2020',
#             path/'ch'/'social'/'2019',
#             path/'ch'/'social'/'2020']

In [7]:
#Appending into dataframe

def get_df(path_list):
    df = []
    first = True
    for directory in tqdm(path_list, desc='Years'):
        f_list = os.listdir(directory)
        csv_files = [directory/f for f in f_list if f.endswith('.csv')]
        for csv in tqdm(csv_files, desc='Days'):
            if first:
                df = pd.read_csv(csv, dtype={'cbg': object})
                first = False
            else:
                df = df.append(pd.read_csv(csv, dtype={'cbg': object}), ignore_index=True)
    return df.sort_values(by=['date_range_start'])

In [8]:
%%time

df = get_df(path_list)



CPU times: user 9min 7s, sys: 14.1 s, total: 9min 21s
Wall time: 14min 41s


In [9]:
df

,Unnamed: 0,cbg,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home,mean_home_dwell_time,mean_non_home_dwell_time,mean_distance_traveled_from_home
1965667,74661,170319900000,2020-01-01T00:00:00-05:00,2020-01-02T00:00:00-05:00,19,1947.0,"{""0"":3,""2001-8000"":2,""1-1000"":2,""1001-2000"":6,...","{""1001-2000"":73,""2001-8000"":248,""8001-16000"":6...",1,0,"{""721-1080"":4,""<60"":11,""361-720"":1,""61-360"":3}","[1,1,2,1,1,1,1,2,1,3,2,1,2,1,1,2,3,2,2,1,4,2,3,2]",1,1,"{""170310402021"":1,""170314201001"":1,""1703143060...",1,74,92,"{""21-45"":1,""46-60"":1,""721-840"":1,""301-360"":4,""...",0,"{""0-25"":11,""76-100"":3,""51-75"":1}",NaN,NaN,NaN
1965643,73995,170311002005,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,54,9541.0,"{""16001-50000"":7,""0"":19,"">50000"":5,""2001-8000""...","{""16001-50000"":195,"">50000"":334,""<1000"":19,""20...",18,709,"{""721-1080"":5,""361-720"":12,""61-360"":7,""<60"":9,...","[16,18,23,24,27,24,24,24,26,25,25,28,28,29,32,...",1,1,"{""170978648011"":3,""170318043083"":1,""1703180660...",1,94,93,"{""21-45"":4,""481-540"":3,""46-60"":3,""721-840"":3,""...",87,"{""26-50"":6,""51-75"":8,""0-25"":9,"">100"":1,""76-100...",NaN,NaN,NaN
1965644,73996,170312214003,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,56,4719.0,"{""16001-50000"":2,""0"":25,"">50000"":2,""2001-8000""...","{""16001-50000"":44,"">50000"":135,""<1000"":80,""200...",25,816,"{""721-1080"":5,""361-720"":9,""61-360"":7,""<60"":7,""...","[17,20,23,26,27,32,31,33,30,32,34,34,32,33,32,...",2,1,"{""170318150002"":1,""170312411002"":1,""1703122220...",1,14,108,"{""21-45"":5,""481-540"":1,""541-600"":2,""721-840"":2...",97,"{""26-50"":1,""51-75"":3,""0-25"":8,"">100"":6,""76-100...",NaN,NaN,NaN
1965645,73997,170318241074,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,123,4947.0,"{""16001-50000"":12,""0"":44,"">50000"":4,""2001-8000...","{""16001-50000"":73,"">50000"":302,""<1000"":143,""20...",44,915,"{""721-1080"":23,""361-720"":17,""61-360"":12,""<60"":...","[73,78,81,82,85,83,84,77,84,83,79,75,73,68,70,...",2,1,"{""170318253031"":1,""170319801001"":2,""1703182100...",1,78,193,"{""21-45"":2,""481-540"":1,""541-600"":1,""46-60"":8,""...",93,"{""26-50"":7,""51-75"":13,""0-25"":21,"">100"":1,""76-1...",NaN,NaN,NaN
1965646,73998,170318263012,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,72,10907.0,"{""16001-50000"":7,""0"":26,"">50000"":5,""2001-8000""...","{""16001-50000"":60,"">50000"":20,""<1000"":508,""200...",25,870,"{""721-1080"":10,""361-720"":7,""61-360"":11,""<60"":1...","[31,34,36,36,37,39,37,37,38,38,36,36,39,38,38,...",3,1,"{""170318273003"":1,""170319801001"":1,""1703182740...",1,28,118,"{""21-45"":6,""46-60"":2,""721-840"":3,""1201-1320"":4...",96,"{""0-25"":15,""76-100"":40,""51-75"":5,""26-50"":5}",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2353875,149545,170898503022,2020-12-31T00:00:00-06:00,2021-01-01T00:00:00-06:00,113,1568.0,"{""16001-50000"":14,""0"":29,"">50000"":5,""2001-8000...","{""16001-50000"":104,"">50000"":241,""<1000"":259,""2...",26,960,"{""721-1080"":26,""361-720"":9,""61-360"":4,""<60"":32...","[63,63,64,63,62,65,63,64,54,53,56,53,53,53,56,...",10,1,"{""171118713064"":1,""170898501052"":1,""1708985050...",1,127,227,"{""21-45"":8,""481-540"":3,""541-600"":1,""721-840"":3...",78,"{""0-25"":33,""76-100"":58,""51-75"":10,""26-50"":3}",740.0,268.0,8230.0
2353876,149548,171978802042,2020-12-31T00:00:00-06:00,2021-01-01T00:00:00-06:00,241,2562.0,"{""16001-50000"":27,""0"":87,"">50000"":13,""2001-800...",

In [10]:
[print(c) for c in df.columns];

Unnamed: 0
cbg
date_range_start
date_range_end
device_count
distance_traveled_from_home
bucketed_distance_traveled
median_dwell_at_bucketed_distance_traveled
completely_home_device_count
median_home_dwell_time
bucketed_home_dwell_time
at_home_by_each_hour
part_time_work_behavior_devices
full_time_work_behavior_devices
destination_cbgs
delivery_behavior_devices
median_non_home_dwell_time
candidate_device_count
bucketed_away_from_home_time
median_percentage_time_home
bucketed_percentage_time_home
mean_home_dwell_time
mean_non_home_dwell_time
mean_distance_traveled_from_home


In [11]:
# Selecting the columsn that we need for the analysis
# col = ['date_range_start',
#        'cbg',
#        'device_count',  # Total active devices
#        'completely_home_device_count',
#        'part_time_work_behavior_devices',
#        'full_time_work_behavior_devices',
#        'delivery_behavior_devices',
#        'median_percentage_time_home',
#        'median_home_dwell_time',
#        'mean_home_dwell_time',
#        'median_non_home_dwell_time',
#        'mean_non_home_dwell_time',
#        'distance_traveled_from_home', # Median Distance traveled
#        'mean_distance_traveled_from_home']
col = ['date_range_start',
       'cbg',
       'device_count',  # Total active devices
       'completely_home_device_count',
       'part_time_work_behavior_devices',
       'full_time_work_behavior_devices']
devices_mobility = df[col]

In [12]:
#Renaming columns

devices_mobility = devices_mobility.rename(columns={'date_range_start':'date','device_count':'total', 'completely_home_device_count':'completely_home','part_time_work_behavior_devices':'part_time_work', 'full_time_work_behavior_devices':'full_time_work'})
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work
1965667,2020-01-01T00:00:00-05:00,170319900000,19,1,1,1
1965643,2020-01-01T00:00:00-06:00,170311002005,54,18,1,1
1965644,2020-01-01T00:00:00-06:00,170312214003,56,25,2,1
1965645,2020-01-01T00:00:00-06:00,170318241074,123,44,2,1
1965646,2020-01-01T00:00:00-06:00,170318263012,72,25,3,1
...,...,...,...,...,...,...
2353875,2020-12-31T00:00:00-06:00,170898503022,113,26,10,1
2353876,2020-12-31T00:00:00-06:00,171978802042,241,86,14,10
2353877,2020-12-31T00:00:00-06:00,171978838081,78,29,2,1
2354426,2020-12-31T00:00:00-06:00,170316120002,29,16,1,1


In [13]:
# devices_mobility

In [14]:
#Listing out the device count columns

device_columns = ['completely_home','part_time_work','full_time_work']

In [15]:
#Calculating percentage for all the device columns
for column in device_columns :
    devices_mobility['percentage_'+column]= devices_mobility.apply(lambda row: row[column]/row['total']*100.0 if row['total']!=0.0 else 0.0, axis=1)
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work
1965667,2020-01-01T00:00:00-05:00,170319900000,19,1,1,1,5.263158,5.263158,5.263158
1965643,2020-01-01T00:00:00-06:00,170311002005,54,18,1,1,33.333333,1.851852,1.851852
1965644,2020-01-01T00:00:00-06:00,170312214003,56,25,2,1,44.642857,3.571429,1.785714
1965645,2020-01-01T00:00:00-06:00,170318241074,123,44,2,1,35.772358,1.626016,0.813008
1965646,2020-01-01T00:00:00-06:00,170318263012,72,25,3,1,34.722222,4.166667,1.388889
...,...,...,...,...,...,...,...,...,...
2353875,2020-12-31T00:00:00-06:00,170898503022,113,26,10,1,23.008850,8.849558,0.884956
2353876,2020-12-31T00:00:00-06:00,171978802042,241,86,14,10,35.684647,5.809129,4.149378
2353877,2020-12-31T00:00:00-06:00,171978838081,78,29,2,1,37.179487,2.564103,1.282051
2354426,2020-12-31T00:00:00-06:00,170316120002,29,16,1,1,55.172414,3.448276,3.448276


In [16]:
#Normalising the percentage columns

for column in device_columns:
        devices_mobility['norm_'+column] = (devices_mobility['percentage_'+ column] - devices_mobility['percentage_'+column].mean()) / devices_mobility['percentage_'+column].std()

devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
1965667,2020-01-01T00:00:00-05:00,170319900000,19,1,1,1,5.263158,5.263158,5.263158,-2.393167,-0.359531,0.064910
1965643,2020-01-01T00:00:00-06:00,170311002005,54,18,1,1,33.333333,1.851852,1.851852,0.038678,-1.054165,-0.724282
1965644,2020-01-01T00:00:00-06:00,170312214003,56,25,2,1,44.642857,3.571429,1.785714,1.018473,-0.704013,-0.739583
1965645,2020-01-01T00:00:00-06:00,170318241074,123,44,2,1,35.772358,1.626016,0.813008,0.249982,-1.100151,-0.964615
1965646,2020-01-01T00:00:00-06:00,170318263012,72,25,3,1,34.722222,4.166667,1.388889,0.159004,-0.582806,-0.831387
...,...,...,...,...,...,...,...,...,...,...,...,...
2353875,2020-12-31T00:00:00-06:00,170898503022,113,26,10,1,23.008850,8.849558,0.884956,-0.855778,0.370757,-0.947970
2353876,2020-12-31T00:00:00-06:00,171978802042,241,86,14,10,35.684647,5.809129,4.149378,0.242383,-0.248356,-0.192759
2353877,2020-12-31T00:00:00-06:00,171978838081,78,29,2,1,37.179487,2.564103,1.282051,0.371888,-0.909132,-0.856103
2354426,2020-12-31T00:00:00-06:00,170316120002,29,16,1,1,55.172414,3.448276,3.448276,1.930695,-0.729090,-0.354956


In [17]:
#Converting datetime format
devices_mobility['date'] = pd.to_datetime(devices_mobility['date'],utc= True)
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
1965667,2020-01-01 05:00:00+00:00,170319900000,19,1,1,1,5.263158,5.263158,5.263158,-2.393167,-0.359531,0.064910
1965643,2020-01-01 06:00:00+00:00,170311002005,54,18,1,1,33.333333,1.851852,1.851852,0.038678,-1.054165,-0.724282
1965644,2020-01-01 06:00:00+00:00,170312214003,56,25,2,1,44.642857,3.571429,1.785714,1.018473,-0.704013,-0.739583
1965645,2020-01-01 06:00:00+00:00,170318241074,123,44,2,1,35.772358,1.626016,0.813008,0.249982,-1.100151,-0.964615
1965646,2020-01-01 06:00:00+00:00,170318263012,72,25,3,1,34.722222,4.166667,1.388889,0.159004,-0.582806,-0.831387
...,...,...,...,...,...,...,...,...,...,...,...,...
2353875,2020-12-31 06:00:00+00:00,170898503022,113,26,10,1,23.008850,8.849558,0.884956,-0.855778,0.370757,-0.947970
2353876,2020-12-31 06:00:00+00:00,171978802042,241,86,14,10,35.684647,5.809129,4.149378,0.242383,-0.248356,-0.192759
2353877,2020-12-31 06:00:00+00:00,171978838081,78,29,2,1,37.179487,2.564103,1.282051,0.371888,-0.909132,-0.856103
2354426,2020-12-31 06:00:00+00:00,170316120002,29,16,1,1,55.172414,3.448276,3.448276,1.930695,-0.729090,-0.354956


In [18]:
# Normalize time
devices_mobility['date'] = devices_mobility['date'].dt.normalize()
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
1965667,2020-01-01 00:00:00+00:00,170319900000,19,1,1,1,5.263158,5.263158,5.263158,-2.393167,-0.359531,0.064910
1965643,2020-01-01 00:00:00+00:00,170311002005,54,18,1,1,33.333333,1.851852,1.851852,0.038678,-1.054165,-0.724282
1965644,2020-01-01 00:00:00+00:00,170312214003,56,25,2,1,44.642857,3.571429,1.785714,1.018473,-0.704013,-0.739583
1965645,2020-01-01 00:00:00+00:00,170318241074,123,44,2,1,35.772358,1.626016,0.813008,0.249982,-1.100151,-0.964615
1965646,2020-01-01 00:00:00+00:00,170318263012,72,25,3,1,34.722222,4.166667,1.388889,0.159004,-0.582806,-0.831387
...,...,...,...,...,...,...,...,...,...,...,...,...
2353875,2020-12-31 00:00:00+00:00,170898503022,113,26,10,1,23.008850,8.849558,0.884956,-0.855778,0.370757,-0.947970
2353876,2020-12-31 00:00:00+00:00,171978802042,241,86,14,10,35.684647,5.809129,4.149378,0.242383,-0.248356,-0.192759
2353877,2020-12-31 00:00:00+00:00,171978838081,78,29,2,1,37.179487,2.564103,1.282051,0.371888,-0.909132,-0.856103
2354426,2020-12-31 00:00:00+00:00,170316120002,29,16,1,1,55.172414,3.448276,3.448276,1.930695,-0.729090,-0.354956


In [19]:
# Check the number of rows for each CBG
print('Min: {}\nMax: {}'.format(min(set(devices_mobility.groupby('cbg').count()['date'])), max(set(devices_mobility.groupby('cbg').count()['date']))))

Min: 7
Max: 366


In [20]:
for col in devices_mobility.columns:
    print('\n')
    print(col)
#     if devices_mobility[col].dtype
#     print(devices_mobility[col].dtype)
    print('Min: {}\nMax: {}'.format(devices_mobility[col].min(), devices_mobility[col].max()))



date
Min: 2020-01-01 00:00:00+00:00
Max: 2020-12-31 00:00:00+00:00


cbg
Min: 170310101001
Max: 550599900000


total
Min: 5
Max: 2796


completely_home
Min: 1
Max: 1328


part_time_work
Min: 1
Max: 306


full_time_work
Min: 1
Max: 249


percentage_completely_home
Min: 0.5434782608695652
Max: 92.0


percentage_part_time_work
Min: 0.12610340479192939
Max: 60.0


percentage_full_time_work
Min: 0.07230657989877079
Max: 57.14285714285714


norm_completely_home
Min: -2.8020534926009653
Max: 5.1212339679434615


norm_part_time_work
Min: -1.4055743023917877
Max: 10.786372721802703


norm_full_time_work
Min: -1.135973135461557
Max: 12.067073989321642


In [21]:
# Print all Null and NaN values
print('We have {} null values.'.format(len(devices_mobility[devices_mobility.isna().any(axis=1)])))

We have 0 null values.


In [22]:
# Grouping based on day
grouped_dm = devices_mobility.groupby(devices_mobility['date']).mean()
grouped_dm

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
date,,,,,,,,,,
2020-01-01 00:00:00+00:00,83.040264,26.278846,4.232873,1.570913,31.487162,4.949419,2.298667,-0.121264,-0.423417,-0.620913
2020-01-02 00:00:00+00:00,79.955973,22.011119,7.696619,5.833509,27.869436,9.477895,7.402454,-0.434684,0.498704,0.559828
2020-01-03 00:00:00+00:00,81.248498,20.842999,7.750601,5.202674,25.764951,9.248650,6.429959,-0.617004,0.452023,0.334845
2020-01-04 00:00:00+00:00,80.970398,23.618182,6.134485,2.630353,29.659083,7.420016,3.587309,-0.279638,0.079664,-0.322791
2020-01-05 00:00:00+00:00,81.472514,26.920547,5.892010,2.280565,33.435897,7.241368,3.258508,0.047564,0.043286,-0.398858
...,...,...,...,...,...,...,...,...,...,...
2020-12-27 00:00:00+00:00,78.281100,31.094802,4.985276,2.112230,37.087833,6.786157,3.484098,0.363947,-0.049407,-0.346669
2020-12-28 00:00:00+00:00,78.720637,27.908231,5.797687,3.678432,32.893410,7.667484,5.275423,0.000566,0.130055,0.067747
2020-12-29 00:00:00+00:00,78.567673,28.342947,5.726604,3.493315,33.284867,7.477308,4.978050,0.034479,0.091330,-0.001049


In [23]:
grouped_dm.reset_index()

,date,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
0,2020-01-01 00:00:00+00:00,83.040264,26.278846,4.232873,1.570913,31.487162,4.949419,2.298667,-0.121264,-0.423417,-0.620913
1,2020-01-02 00:00:00+00:00,79.955973,22.011119,7.696619,5.833509,27.869436,9.477895,7.402454,-0.434684,0.498704,0.559828
2,2020-01-03 00:00:00+00:00,81.248498,20.842999,7.750601,5.202674,25.764951,9.248650,6.429959,-0.617004,0.452023,0.334845
3,2020-01-04 00:00:00+00:00,80.970398,23.618182,6.134485,2.630353,29.659083,7.420016,3.587309,-0.279638,0.079664,-0.322791
4,2020-01-05 00:00:00+00:00,81.472514,26.920547,5.892010,2.280565,33.435897,7.241368,3.258508,0.047564,0.043286,-0.398858
...,...,...,...,...,...,...,...,...,...,...,...
361,2020-12-27 00:00:00+00:00,78.281100,31.094802,4.985276,2.112230,37.087833,6.786157,3.484098,0.363947,-0.049407,-0.346669
362,2020-12-28 00:00:00+00:00,78.720637,27.908231,5.797687,3.678432,32.893410,7.667484,5.275423,0.000566,0.130055,0.067747
363,2020-12-29 00:00:00+00:00,78.567673,28.342947,5.726604,3.493315,33.284867,7.477308,4.978050,0.034479,0.091330,-0.001049
364,2020-12-30 00:00:00+00:00,78.543939,28.203996,5.328376,3.286616,32.694713,6.924717,4.671759,-0.016648,-0.021192,-0.071908


In [24]:
#Visualising mobility

title = '{} {}'.format(city,year)
alt.Chart(grouped_dm.reset_index()).mark_line().encode(
    x=alt.X('date', title='Date'),
    y=alt.Y(alt.repeat('row'), type='quantitative')
).properties(
    width=300,
    height=250
).repeat(
    row=['percentage_completely_home','percentage_part_time_work','percentage_full_time_work']
).interactive()

alt.RepeatChart(...)

![ch-daily-2020](https://github.com/chouhandiksha/bigdataproject/raw/main/media/social-dist/ch-daily-2020.png)

In [25]:
#Adding month column for further usage
devices_mobility['month'] = devices_mobility['date'].dt.month
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work,month
1965667,2020-01-01 00:00:00+00:00,170319900000,19,1,1,1,5.263158,5.263158,5.263158,-2.393167,-0.359531,0.064910,1
1965643,2020-01-01 00:00:00+00:00,170311002005,54,18,1,1,33.333333,1.851852,1.851852,0.038678,-1.054165,-0.724282,1
1965644,2020-01-01 00:00:00+00:00,170312214003,56,25,2,1,44.642857,3.571429,1.785714,1.018473,-0.704013,-0.739583,1
1965645,2020-01-01 00:00:00+00:00,170318241074,123,44,2,1,35.772358,1.626016,0.813008,0.249982,-1.100151,-0.964615,1
1965646,2020-01-01 00:00:00+00:00,170318263012,72,25,3,1,34.722222,4.166667,1.388889,0.159004,-0.582806,-0.831387,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2353875,2020-12-31 00:00:00+00:00,170898503022,113,26,10,1,23.008850,8.849558,0.884956,-0.855778,0.370757,-0.947970,12
2353876,2020-12-31 00:00:00+00:00,171978802042,241,86,14,10,35.684647,5.809129,4.149378,0.242383,-0.248356,-0.192759,12
2353877,2020-12-31 00:00:00+00:00,171978838081,78,29,2,1,37.179487,2.564103,1.282051,0.371888,-0.909132,-0.856103,12
2354426,2020-12-31 00:00:00+00:00,170316120002,29,16,1,1,55.172414,3.448276,3.448276,1.930695,-0.729090,-0.354956,12


In [26]:
#Grouping data on basis of month value

grouped_dm = devices_mobility.groupby([devices_mobility['month']]).mean()
grouped_dm

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
month,,,,,,,,,,
1,75.731500,18.397066,8.886829,6.958972,25.069652,11.262816,9.039253,-0.677241,0.862162,0.938495
2,86.902653,21.612731,9.750468,7.158128,25.550680,10.792575,8.210081,-0.635568,0.766408,0.746669
3,80.766416,26.799246,6.104262,4.539385,34.516746,7.293131,5.663679,0.141203,0.053827,0.157569
4,76.550880,35.277100,2.982271,2.067550,45.895039,4.171168,3.162395,1.126955,-0.581890,-0.421093
5,80.216564,32.497764,3.641978,2.254658,40.673505,4.692580,3.256675,0.674590,-0.475716,-0.399282
6,80.459913,27.358232,4.633519,2.675066,34.013652,5.739939,3.688378,0.097617,-0.262445,-0.299409
7,79.411579,25.842550,4.725777,2.678400,31.581232,6.021827,3.837431,-0.113114,-0.205045,-0.264926
8,84.667192,28.890274,5.019867,2.887332,29.878317,6.353675,4.045162,-0.260645,-0.137472,-0.216869
9,80.486525,24.147559,5.211021,3.274483,30.198707,6.570086,4.465736,-0.232888,-0.093405,-0.119571


In [27]:
d = []
for col in ['percentage_completely_home','percentage_part_time_work','percentage_full_time_work']:
        for m in range(1,13):
            d.append({'month': m, 'column': col, 'value': grouped_dm[col][m]})
d = alt.Data(values=d)

In [28]:
title = '{} {}'.format(city,year)
alt.Chart(d, title=title).mark_line().encode(
    x=alt.X('month:O', title='Month'),
    y=alt.Y('value:Q', type='quantitative', title='Percentage'),
    color='column:N'
).properties(
    width=300,
    height=250
).interactive()

alt.Chart(...)

![ch-monthly-2020](https://github.com/chouhandiksha/bigdataproject/raw/main/media/social-dist/ch-monthly-2020.png)

In [29]:
grouped_daily = devices_mobility.groupby([devices_mobility['date']]).mean()
grouped_daily

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work,month
date,,,,,,,,,,,
2020-01-01 00:00:00+00:00,83.040264,26.278846,4.232873,1.570913,31.487162,4.949419,2.298667,-0.121264,-0.423417,-0.620913,1.0
2020-01-02 00:00:00+00:00,79.955973,22.011119,7.696619,5.833509,27.869436,9.477895,7.402454,-0.434684,0.498704,0.559828,1.0
2020-01-03 00:00:00+00:00,81.248498,20.842999,7.750601,5.202674,25.764951,9.248650,6.429959,-0.617004,0.452023,0.334845,1.0
2020-01-04 00:00:00+00:00,80.970398,23.618182,6.134485,2.630353,29.659083,7.420016,3.587309,-0.279638,0.079664,-0.322791,1.0
2020-01-05 00:00:00+00:00,81.472514,26.920547,5.892010,2.280565,33.435897,7.241368,3.258508,0.047564,0.043286,-0.398858,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27 00:00:00+00:00,78.281100,31.094802,4.985276,2.112230,37.087833,6.786157,3.484098,0.363947,-0.049407,-0.346669,12.0
2020-12-28 00:00:00+00:00,78.720637,27.908231,5.797687,3.678432,32.893410,7.667484,5.275423,0.000566,0.130055,0.067747,12.0
2020-12-29 00:00:00+00:00,78.567673,28.342947,5.726604,3.493315,33.284867,7.477308,4.978050,0.034479,0.091330,-0.001049,12.0


In [30]:
#Visualising mobility

title = '{} {}'.format(city,year)
alt.Chart(grouped_daily.reset_index()).mark_line().encode(
    x=alt.X('date', title='Date'),
    y=alt.Y(alt.repeat('row'), type='quantitative')
).properties(
    width=300,
    height=250
).repeat(
    row=['percentage_completely_home','percentage_part_time_work','percentage_full_time_work']
).interactive()

alt.RepeatChart(...)

In [31]:
grouped_daily

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work,month
date,,,,,,,,,,,
2020-01-01 00:00:00+00:00,83.040264,26.278846,4.232873,1.570913,31.487162,4.949419,2.298667,-0.121264,-0.423417,-0.620913,1.0
2020-01-02 00:00:00+00:00,79.955973,22.011119,7.696619,5.833509,27.869436,9.477895,7.402454,-0.434684,0.498704,0.559828,1.0
2020-01-03 00:00:00+00:00,81.248498,20.842999,7.750601,5.202674,25.764951,9.248650,6.429959,-0.617004,0.452023,0.334845,1.0
2020-01-04 00:00:00+00:00,80.970398,23.618182,6.134485,2.630353,29.659083,7.420016,3.587309,-0.279638,0.079664,-0.322791,1.0
2020-01-05 00:00:00+00:00,81.472514,26.920547,5.892010,2.280565,33.435897,7.241368,3.258508,0.047564,0.043286,-0.398858,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27 00:00:00+00:00,78.281100,31.094802,4.985276,2.112230,37.087833,6.786157,3.484098,0.363947,-0.049407,-0.346669,12.0
2020-12-28 00:00:00+00:00,78.720637,27.908231,5.797687,3.678432,32.893410,7.667484,5.275423,0.000566,0.130055,0.067747,12.0
2020-12-29 00:00:00+00:00,78.567673,28.342947,5.726604,3.493315,33.284867,7.477308,4.978050,0.034479,0.091330,-0.001049,12.0


In [32]:
grouped_rolling = grouped_daily.rolling(10).mean()[9:]
grouped_rolling

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work,month
date,,,,,,,,,,,
2020-01-10 00:00:00+00:00,80.154337,21.054647,8.501939,6.583628,26.735367,10.257467,8.142222,-0.532933,0.657446,0.730970,1.0
2020-01-11 00:00:00+00:00,79.655133,20.856585,8.709796,6.696398,26.740477,10.533763,8.290843,-0.532490,0.713707,0.765353,1.0
2020-01-12 00:00:00+00:00,79.499605,21.182862,8.532878,6.333660,27.252211,10.318505,7.873643,-0.488157,0.669875,0.668835,1.0
2020-01-13 00:00:00+00:00,78.991224,20.766857,8.771176,6.765730,26.959106,10.631389,8.393924,-0.513550,0.733586,0.789200,1.0
2020-01-14 00:00:00+00:00,78.372213,20.021314,9.235350,7.508150,26.276195,11.268327,9.320613,-0.572713,0.863284,1.003586,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27 00:00:00+00:00,74.954375,26.284264,5.307952,2.699469,32.226328,7.211057,4.216698,-0.057226,0.037114,-0.177185,12.0
2020-12-28 00:00:00+00:00,75.189134,26.642364,5.300401,2.688135,32.564865,7.224701,4.224603,-0.027898,0.039892,-0.175356,12.0
2020-12-29 00:00:00+00:00,75.445390,26.980806,5.370282,2.808621,32.863587,7.293078,4.350067,-0.002018,0.053816,-0.146330,12.0


In [33]:
#Visualising mobility

title = '{} {}'.format(city,year)
alt.Chart(grouped_rolling.reset_index()).mark_line().encode(
    x=alt.X('date', title='Date'),
    y=alt.Y(alt.repeat('row'), type='quantitative')
).properties(
    width=300,
    height=250
).repeat(
    row=['percentage_completely_home','percentage_part_time_work','percentage_full_time_work']
).interactive()

alt.RepeatChart(...)

In [34]:
filename = 'drive/MyDrive/big-data-project/sketch/justin/{}_social_summary_{}.csv'.format(city_directory, 2019)
stat_2019 = pd.read_csv(filename, index_col=0)
stat_2019

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
count,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06,2.430227e+06
mean,9.504381e+01,2.983398e+01,8.549604e+00,5.511380e+00,3.096627e+01,8.979337e+00,5.998589e+00,-3.668441e-13,1.189522e-12,-6.917475e-14
std,1.119792e+02,4.251987e+01,1.018083e+01,7.564772e+00,9.897878e+00,5.194712e+00,4.761510e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,5.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.944444e-01,1.718213e-01,6.067961e-02,-3.058415e+00,-1.695477e+00,-1.247064e+00
25%,5.200000e+01,1.500000e+01,3.000000e+00,1.000000e+00,2.413793e+01,4.918033e+00,2.150538e+00,-6.898786e-01,-7.818151e-01,-8.081579e-01
50%,7.200000e+01,2.200000e+01,6.000000e+00,3.000000e+00,3.043478e+01,8.333333e+00,4.597701e+00,-5.369663e-02,-1.243580e-01,-2.942109e-01
75%,1.030000e+02,3.300000e+01,1.100000e+01,7.000000e+00,3.723404e+01,1.223404e+01,8.661417e+00,6.332446e-01,6.265420e-01,5.592402e-01
max,9.089000e+03,3.637000e+03,5.040000e+02,2.440000e+02,9.487179e+01,6.666667e+01,5.714286e+01,6.456488e+00,1.110501e+01,1.074119e+01


In [35]:
stat_2019.loc['mean','percentage_completely_home']

30.966265289442486

In [36]:
grouped_rolling['percentage_completely_home_deviation'] = grouped_rolling['percentage_completely_home'] - stat_2019.loc['mean','percentage_completely_home']
grouped_rolling

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work,month,percentage_completely_home_deviation
date,,,,,,,,,,,,
2020-01-10 00:00:00+00:00,80.154337,21.054647,8.501939,6.583628,26.735367,10.257467,8.142222,-0.532933,0.657446,0.730970,1.0,-4.230898
2020-01-11 00:00:00+00:00,79.655133,20.856585,8.709796,6.696398,26.740477,10.533763,8.290843,-0.532490,0.713707,0.765353,1.0,-4.225788
2020-01-12 00:00:00+00:00,79.499605,21.182862,8.532878,6.333660,27.252211,10.318505,7.873643,-0.488157,0.669875,0.668835,1.0,-3.714055
2020-01-13 00:00:00+00:00,78.991224,20.766857,8.771176,6.765730,26.959106,10.631389,8.393924,-0.513550,0.733586,0.789200,1.0,-4.007159
2020-01-14 00:00:00+00:00,78.372213,20.021314,9.235350,7.508150,26.276195,11.268327,9.320613,-0.572713,0.863284,1.003586,1.0,-4.690071
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27 00:00:00+00:00,74.954375,26.284264,5.307952,2.699469,32.226328,7.211057,4.216698,-0.057226,0.037114,-0.177185,12.0,1.260063
2020-12-28 00:00:00+00:00,75.189134,26.642364,5.300401,2.688135,32.564865,7.224701,4.224603,-0.027898,0.039892,-0.175356,12.0,1.598600
2020-12-29 00:00:00+00:00,75.445390,26.980806,5.370282,2.808621,32.863587,7.293078,4.350067,-0.002018,0.053816,-0.146330,12.0,1.897321


In [37]:
stat_2019.loc['mean','percentage_completely_home']

30.966265289442486

In [38]:
pd.DataFrame({'y':[stat_2019.loc['mean','percentage_completely_home']]})

,y
0,30.966265


In [39]:
#Visualising mobility

title = '{} {}'.format(city,year)
chart = alt.Chart(grouped_rolling.reset_index()).mark_line().encode(
    x=alt.X('date', title='Date'),
    y=alt.Y('percentage_completely_home_deviation', type='quantitative')
).properties(
    width=300,
    height=250
)

line = alt.Chart(pd.DataFrame({'percentage_completely_home_deviation':[0]})).mark_rule(color='#757575',strokeDash=[5,3], size=2).encode(
    y='percentage_completely_home_deviation',
    # color=alt.value("#757575"),
    size=alt.value(2),
)

( line + chart ).interactive()

alt.LayerChart(...)

In [40]:
# Graph Weekly Mobility 
## Difference from avarage
## Grouped by poverty
## Grouped by race

# Graph Daily Mobility